In [1]:
import time
import keras

from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks

import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

2023-04-12 11:20:28.817384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

save_path = "/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/"
model_name = "conv_only_tri_3"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 10

#### define functions ####

def model_cycle():
    inputs = Input(shape=(48, 64, 1))
        
    x = Conv2D(48, kernel_size=(41,64),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # print(f"After first convolutional layer: x.shape = {x.shape}")

    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = Conv1D(48, kernel_size=(11),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After second convolutional layer: x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((24,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After third convolutional layer, x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((12,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # # parallel line 1
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(fx1)
    # fx1 = MaxPooling2D((2,1),padding='same')(fx1)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    
    # # parallel line 2
    # fx2 = Conv2D(48, kernel_size=(11,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    # fx2 = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(fx2)
    # fx2 = MaxPooling2D((2,1),padding='same')(fx2)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    
    # # # Add
    # x1 = Concatenate(axis=-3)([fx1, fx2])
    # x = Add()([x, x1])
    # x = MaxPooling2D((2,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    
    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = LSTM(20, return_sequences=False)(x)
    # x = Dropout(0.2)(x)

    # x = Reshape((1, 24*48))(x)
    x = Flatten()(x)

    outputs = Dense(1, activation='linear')(x)
    # print(outputs.shape)
    network = Model(inputs, outputs)
    network.compile(optimizer='rmsprop',
                    loss='mean_squared_error')
    return network

In [3]:
def triOneHot(sequence):
    code = {"AAA": [0], "AAC": [1], "AAG": [2], "AAT": [3],
            "ACA": [4], "ACC": [5], "ACG": [6], "ACT": [7], 
            "AGA": [8], "AGC": [9], "AGG": [10], "AGT": [11], 
            "ATA": [12], "ATC": [13], "ATG": [14], "ATT": [15], 
            "CAA": [16], "CAC": [17], "CAG": [18], "CAT": [19], 
            "CCA": [20], "CCC": [21], "CCG": [22], "CCT": [23], 
            "CGA": [24], "CGC": [25], "CGG": [26], "CGT": [27], 
            "CTA": [28], "CTC": [29], "CTG": [30], "CTT": [31], 
            "GAA": [32], "GAC": [33], "GAG": [34], "GAT": [35],
            "GCA": [36], "GCC": [37], "GCG": [38], "GCT": [39], 
            "GGA": [40], "GGC": [41], "GGG": [42], "GGT": [43], 
            "GTA": [44], "GTC": [45], "GTG": [46], "GTT": [47], 
            "TAA": [48], "TAC": [49], "TAG": [50], "TAT": [51], 
            "TCA": [52], "TCC": [53], "TCG": [54], "TCT": [55], 
            "TGA": [56], "TGC": [57], "TGG": [58], "TGT": [59], 
            "TTA": [60], "TTC": [61], "TTG": [62], "TTT": [63],   "N": [64]}
    onehot_encoded_seq = []
    for i, _ in enumerate(sequence[:(len(sequence)-2)]):
        onehot_encoded = np.zeros(65)
        onehot_encoded[code[sequence[i:i+3]]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:64])
    return onehot_encoded_seq

In [4]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 5])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 5])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 3])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 3])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 7])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 7])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 0])}")

In [5]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], np.transpose(inv_mat)))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [6]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)
        
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(triOneHot(sequence_nt))
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],48,64,1))
X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = find_c0new(data_cerevisiae_nucle).astype(float)


In [7]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(triOneHot(sequence_nt))
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],48,64,1))
X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = find_c0new(data_random_library).astype(float)


In [8]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(triOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],48,64,1))
X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = find_c0new(data_tiling).astype(float)



In [9]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(triOneHot(sequence_nt))
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],48,64,1))
X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = find_c0new(data_chr5).astype(float)

In [10]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Y5):
    training_X = X5[train_index]
    training_X_reverse = X5_reverse[train_index]
    validation_X = X5[val_index]
    validation_X_reverse = X5_reverse[val_index]
    training_Y = Y5[train_index]
    validation_Y = Y5[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_tiling_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_tiling_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_tiling_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    
detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_tiling.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+ "_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_tiling.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')


2023-04-12 11:21:39.185461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
2310/2317 [============================>.] - ETA: 0s - loss: 0.1514
Epoch 1: val_loss improved from inf to 0.08994, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1.h5
2317/2317 [==============================] - 11s 4ms/step - loss: 0.1512 - val_loss: 0.0899
Epoch 2/10
2313/2317 [============================>.] - ETA: 0s - loss: 0.1013
Epoch 2: val_loss improved from 0.08994 to 0.08457, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1.h5
2317/2317 [==============================] - 10s 4ms/step - loss: 0.1013 - val_loss: 0.0846
Epoch 3/10
2311/2317 [============================>.] - ETA: 0s - loss: 0.0953
Epoch 3: val_loss improved from 0.08457 to 0.08179, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1.h5
2317/2317 [==============================] - 10s 4ms/st

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1/assets


2576/2576 [==============================] - 5s 2ms/step
Epoch 1/10
2311/2317 [============================>.] - ETA: 0s - loss: 0.1507
Epoch 1: val_loss improved from inf to 0.08571, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_2.h5
2317/2317 [==============================] - 13s 5ms/step - loss: 0.1506 - val_loss: 0.0857
Epoch 2/10
2316/2317 [============================>.] - ETA: 0s - loss: 0.1014
Epoch 2: val_loss improved from 0.08571 to 0.07848, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_2.h5
2317/2317 [==============================] - 13s 5ms/step - loss: 0.1014 - val_loss: 0.0785
Epoch 3/10
2317/2317 [==============================] - ETA: 0s - loss: 0.0945
Epoch 3: val_loss improved from 0.07848 to 0.07659, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_2.h

...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_9/assets


2576/2576 [==============================] - 5s 2ms/step
Epoch 1/10
2317/2317 [==============================] - ETA: 0s - loss: 0.1476
Epoch 1: val_loss improved from inf to 0.08623, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_10.h5
2317/2317 [==============================] - 16s 7ms/step - loss: 0.1476 - val_loss: 0.0862
Epoch 2/10
2315/2317 [============================>.] - ETA: 0s - loss: 0.0957
Epoch 2: val_loss improved from 0.08623 to 0.07924, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_10.h5
2317/2317 [==============================] - 16s 7ms/step - loss: 0.0957 - val_loss: 0.0792
Epoch 3/10
2314/2317 [============================>.] - ETA: 0s - loss: 0.0889
Epoch 3: val_loss improved from 0.07924 to 0.07670, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_1

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_tri_3_tiling_10/assets


2576/2576 [==============================] - 4s 2ms/step


In [11]:
display_fits(fits)

Average correlation on tiling: 0.7683891379748434 
Average MSE on tiling: 0.09938749507249059 
Average correlation on random: 0.7385039570879596 
Average MSE on random: 0.06558267344930362 
Average correlation on ChrV: 0.6413963065975763 
Average MSE on ChrV: 0.1716898243019831 
Average correlation on CN: 0.712381398437249 
Average MSE on CN: 0.12278114090550282


In [12]:
first_conv_model = Model(inputs = model.input, outputs = model.layers[1].output)
first_conv_output = first_conv_model.predict(X5)
pd.DataFrame(first_conv_output.reshape(first_conv_output.shape[0], -1)).to_csv(save_path + model_name+"_tiling_tiling_first_conv_output")
first_conv_output_random = first_conv_model.predict(X3)
pd.DataFrame(first_conv_output_random.reshape(first_conv_output_random.shape[0], -1)).to_csv(save_path + model_name+"_tiling_random_first_conv_output")

390/390 [==============================] - 2s 4ms/step


In [13]:
first_conv_weights = model.layers[1].weights[0]
first_conv_biases = model.layers[1].weights[1]
# Format: [Position 0: AAA, AAC, ..., CAA, CAC, ..., TTG, TTT  Position 1: AAA, AAC, ..., CAA, CAC, ..., TTG, TTT,
# ..., Position 20: AAA, AAC, ..., CAA, CAC, ..., TTG, TTT]
pd.DataFrame(array(first_conv_weights).transpose((3,2,0,1)).reshape(first_conv_weights.shape[-1], -1)).to_csv(save_path + model_name+"_tiling_first_conv_kernels")
pd.DataFrame(first_conv_biases).to_csv(save_path + model_name+"_tiling_first_conv_biases")